In [125]:
print('hello')

hello


In [ ]:
# Library import
import requests
import time
import os
from bs4 import BeautifulSoup
import webbrowser
import re
import numpy as np
import pandas as pd
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
class NoItemError(Exception) :
    def __init__(self) :
        super().__init__('상품이 없습니다.')

In [ ]:
##### 전역변수 선언 및 초기화 #####

sid = int(input('시작 일련번호를 입력하세요...')) # 상품 일련번호
N = int(input('종료 일련번호를 입력하세요...'))
ID = input('아이디를 입력하세요...')
PW = input('패스워드를 입력하세요...')
title = 0 # 신발명

folder = 'C:/workspace/shoedata/' # 이미지파일을 저장할 폴더

URL = "https://kream.co.kr/products/" + str(sid) # 제품 상세페이지 url

header = {
    'Accept-Language': 'ko_KR,en;q=0.8', 'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
    }

# 데이터를 저장할 리스트
mydata = []

# 중복 확인을 위한 변수 설정
is_first = True

# 수집에서 제외할 품목의 리스트
excepts = [
    'Pocket', 'Card', 'Shirt', 'Ball Cap', 'Hoodie', 'Jacket',
    'Watch', 'Coat', 'Pants', 'Shirt', 'Cardigan', 'Knit', 'Wallet',
    'Parfum', 'Lego', 'Pouch', 'Parakeet', 'Bag', 'Stove',
    'Table', 'Chair', 'Waistpack', 'Torch', 'Microsoft',
    'Apple', 'Nintendo', 'Sony', 'Dyson', 'Speaker', 'Marshall',
    'Helinox', 'Samsung', 'Logitech', 'IKEA', 'Printer', 'Cassette',
    'Cushion', 'Pendant', '(PS)', '(GS)', '(TD)', 'Sweatshirt', 'Hat',
    'Clutch', 'T-Shirt',
    ]

ExceptSet = set(excepts)


remove_set = ['빠른배송', 'EU', 'US', 'UK']


# 거래내역을 저장할 빈 데이터프레임
trandf = pd.DataFrame()


##### SELENIUM SETTING #####
# Headless option
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36")
# selenium_webdriver 위치 지정
driver = webdriver.Chrome('./chromedriver.exe', options = options)


In [ ]:
def request(url) :
    '''
    웹사이트에 요청을 보내 html 소스를 가져오는 function
    '''
    global sid, URL
    try :
        response = requests.get(URL, headers = header)
    except Exception as e :
        print('error!')
        print(e)
        print('5초 후 재시도...')
        time.sleep(5)
        request(url) # 재귀함수 호출
    if response.status_code != 200 :
        print(response.status_code)
        raise f'${response.status_code}'
        # sid += 1
        # URL = "https://kream.co.kr/products/" + str(sid) # 제품 상세페이지 url
    return response.text # HTML 데이터

In [ ]:
def check_mydata() :
    '''
    데이터 목록을 확인하는 function
    '''
    for element in mydata :
        if not element['is_checked'] : 
            element['is_checked'] = True
            print(f'확인 중 ==> ', element['Name']) # 콘솔에 체크한 데이터 이름을 표시
            
            if not any(x in element['Name'] for x in excepts) :
                # 만약 수집 대상이 아닌(except 변수에 지정된 품목들) 품목이 아니면
                print(f'새로운 아이템 : ', element['Name'])

In [ ]:
def save_img(soup) :
    '''
    상세페이지의 이미지를 저장하는 function
    '''
    imgtemp = soup.find('picture', class_ = 'picture product_img')
    imgtemp = str(soup.find('img')).split()[-1]
    imgURL = re.sub("'", '', imgtemp.split('"')[1])
    print(imgURL)
    path = folder + f'{sid}_{title}.jpg'
    urllib.request.urlretrieve(imgURL, path)
    # print('저장완료했어요')

In [ ]:
def kream_login() :
    '''
    Kream 사이트에 로그인하는 function
    '''
    # access to url
    url = 'https://kream.co.kr/'
    driver.get(url)
    # wait
    time.sleep(3)
    # Login
    # XPATH = '//*[@id="__layout"]/div/div[1]/div[1]/div/ul/li[4]/a'
    login_btn = driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[1]/div[1]/div/ul/li[4]/a')
    login_btn.click()
    driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[2]/div[1]/div/div[1]/div/input').send_keys(ID)
    driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[2]/div[1]/div/div[2]/div/input').send_keys(PW)
    driver.find_element_by_class_name('login_btn_box').click()
    time.sleep(4)


In [ ]:
def transaction_list(url) :
    '''
    최근 거래 내역 50건을 저장하는 function
    '''
    try :
        # Go to product page
        driver.get(url)
        WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="panel1"]/a')))
        # Find a more btn
        # wait.until(EC.element_to_be_clickable(By.XPATH, '//*[@id="panel1"]/a'))
        MorePrice = driver.find_element(By.XPATH, '//*[@id="panel1"]/a')
        # click the btn
        MorePrice.click()
        time.sleep(1)
        # Find a 260 Size
        # WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="__layout"]/div/div[2]/div[1]/div/div[2]/div/div[6]/div/div[2]/div[1]/div[2]/div/div/button')))
        # SizeBtn = driver.find_element_by_xpath('//*[@id="__layout"]/div/div[2]/div[1]/div/div[2]/div/div[6]/div/div[2]/div[1]/div[2]/div/div/button')
        # SizeBtn.click()
        # XPATH = '//*[@id="__layout"]/div/div[2]/div[1]/div/div[2]/div/div[6]/div/div[2]/div[1]/div[2]/div/div/div/div/div[2]/ul/li[11]/a'
        # Size260 = driver.find_element_by_xpath(XPATH)
        # Size260.click()
        # time.sleep(1)
        # Scroll
        # XPATH = '//*[@id="panel1"]/div/div/div[1]/div/div[3]/a' # Sort
        # driver.find_element(By.XPATH, XPATH).click()
        # time.sleep(0.5)
        # Target = driver.find_element(By.XPATH, '//*[@id="panel1"]/div/div/div[2]/div[1]/div[3]')
        # driver.find_element(By.XPATH, XPATH).click()
        # driver.execute_script('argumnents[0].scrollby(0,269)', Target)

        # selenium으로 조작한 페이지를 저장
        html = driver.page_source
        # selenium 작업을 종료
        # driver.quit()
        # bs4 로 처리
        soup = BeautifulSoup(html, 'html.parser')
        # 최근 거래 내역 50 건
        transaction = soup.find_all('div', class_ = 'price_body')[0]
        transaction = transaction.get_text().split()
        transaction = [i for i in transaction if i not in remove_set]

        transaction = [transaction[i:i + 3] for i in range(0, len(transaction), 3)]
        temps = pd.DataFrame(transaction).drop([0,2], axis = 1)
        temps = temps.transpose()
        time.sleep(1.5)
    
    except Exception :
        print('거래 내역 없음')
        temps = pd.DataFrame([None])
    
    return temps


In [ ]:
 # kream에 로그인
kream_login()

# 크롤링 시작
while sid != N :
    try :
        print(sid) # n 번째 품목입니다.

        ##### bs4 객체 생성 #####
        html = request(URL)

        soup = BeautifulSoup(html, 'html.parser')

        ##### 변수값 설정 #####
        brand = soup.find('a', class_ = 'brand').get_text() # 브랜드

        title = soup.find('p', class_ = 'title').get_text() # 이름
        
        titleSet = set(title.split())

        if titleSet & ExceptSet != set() :
            print('대상이 아님')
            sid += 1
            URL = "https://kream.co.kr/products/" + str(sid) # 제품 상세페이지 url
            continue
        

        if ' x ' in title :
            CollabBrand = title.split(' x ')[1].split()[0] # 콜라보레이션 제품이라면 콜라보레이션 브랜드를 저장
        else : 
            CollabBrand = '-'


        # 상품 상세 정보 불러오기
        temp = soup.find_all('dd', class_ = 'product_info')
        details = []
        for detail in temp :
            temp1 = str(detail)
            temp1 = re.sub('<.+?>','',temp1,0).strip()
            details.append(temp1)

        model_num = details[0] # 모델 번호

        release_date = details[1] # 출시일

        color = details[2] # 주로 쓰인 색상

        release_price = details[3] # 출시 가격

        # 최근 5 체결가 평균
        recent_purchase = soup.find('div', id = 'panel1').get_text()
        recent_purchase = recent_purchase.split()
        recent_purchase = [won for won in recent_purchase if '원' in won]
        recent_purchase = " ".join(recent_purchase)
        recent_purchase = re.sub('[^0-9 ]','', recent_purchase)
        recent_purchase = recent_purchase.split()
        temp = list(map(int, recent_purchase))
        temp = np.array(temp)
        mean_purchase = np.mean(temp)
        
        # 이미지 저장
        save_img(soup)

        ##### mydata 에 수집한 정보 저장 #####
        is_new_item = True # 새로운 상품인지 여부를 표현하는 변수
        temp = {
            'brand' : brand,
            'CollabBrand' : CollabBrand,
            'Name' : title,
            'Model Num' : model_num,
            'Release Date' : release_date,
            'Color' : color,
            'Release Price' : release_price,
            'Average Price' : mean_purchase,
            'is_checked' : False,
            'URL' : URL,
        }

        for element in mydata :
            if element['URL'] == temp['URL'] :
                is_new_item = False
                break

        if is_new_item :
            mydata.append(temp)
        check_mydata()
        
        time.sleep(2.2)

        # 최근 50 거래
        transaction = transaction_list(URL)
        trandf = pd.concat([trandf, transaction], ignore_index = True)

        sid += 1
        URL = "https://kream.co.kr/products/" + str(sid) # 제품 상세페이지 url
    except Exception as e :
        print('error!')
        sid += 1
        URL = "https://kream.co.kr/products/" + str(sid) # 제품 상세페이지 url
        continue


driver.quit()
print('드라이버를 종료합니다.')
mydata = pd.DataFrame(mydata)
mydata = pd.concat([mydata, trandf], axis = 1)
mydata.to_csv('./KREAMDATA.csv', index = False)
print('데이터를 저장했습니다.')
